# Capstone Project - The Battle of Neighborhoods (Week 2)

## 1. A description of the problem and a discussion of the background.**

I am interested in the question about looking for the best location for coffee shop. I am in Ching Mai ,Thailand. This city have a coffee culture. there are many the coffee shop in here. There are some shop success. Too many had fail so, I was Thinking about location might be the main feature of sucess.

The main problem is what is the best location for opene new cafe? Are ther relationship between other shop with cafe such as restaurant, hotel, shopping mall?

## 2. A description of the data and how it will be used to solve the problem.**

The data from Foursquare. By use API search the coffee shop in Ching Mai

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 3. Download and Explore Dataset

In [2]:
data = pd.read_csv (r'/Users/mac/distric.csv')   #read the csv file (put 'r' before the path string to address any special characters in the path, such as '\'). Don't forget to put the file name at the end of the path + ".csv"
data

,AD_LEVEL,TA_ID,TAMBON_T,TAMBON_E,AM_ID,AMPHOE_T,AMPHOE_E,CH_ID,CHANGWAT_T,CHANGWAT_E,LAT,LONG
0,4,500102,ต. พระสิงห์,Phra Sing,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.785,98.985
1,4,500103,ต. หายยา,Hai Ya,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.777,98.984
2,4,500104,ต. ช้างม่อย,Chang Moi,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.796,98.996
3,4,500115,ต. ป่าตัน,Pa Tan,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.811,98.999
4,4,500101,ต. ศรีภูมิ,Si Phum,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.796,98.985
5,4,500116,ต. สันผีเสื้อ,San Phisuea,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.841,98.998
6,4,500113,ต. หนองป่าครั่ง,Nong Pa Khrang,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.788,99.034
7,4,500105,ต. ช้างคลาน,Chang Khlan,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.774,98.996
8,4,500111,ต. หนองหอย,Nong Hoi,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.758,99.015
9,4,500114,ต. ฟ้าฮ่าม,Fa Ham,5001,อ. เมืองเชียงใหม่,Mueang Chiang Mai,50,จ. เชียงใหม่,Chiang Mai,18.816,99.008


In [3]:
df = pd.DataFrame(data, columns= ['TAMBON_T','TAMBON_E','LAT','LONG'])
print (df)

           TAMBON_T        TAMBON_E     LAT    LONG
0       ต. พระสิงห์       Phra Sing  18.785  98.985
1          ต. หายยา          Hai Ya  18.777  98.984
2       ต. ช้างม่อย       Chang Moi  18.796  98.996
3         ต. ป่าตัน          Pa Tan  18.811  98.999
4        ต. ศรีภูมิ         Si Phum  18.796  98.985
5     ต. สันผีเสื้อ     San Phisuea  18.841  98.998
6   ต. หนองป่าครั่ง  Nong Pa Khrang  18.788  99.034
7       ต. ช้างคลาน     Chang Khlan  18.774  98.996
8        ต. หนองหอย        Nong Hoi  18.758  99.015
9        ต. ฟ้าฮ่าม          Fa Ham  18.816  99.008
10        ต. วัดเกต         Wat Ket  18.785  99.011
11       ต. ท่าศาลา        Tha Sala  18.770  99.034
12        ต. ป่าแดด         Pa Daet  18.742  98.978
13     ต. ช้างเผือก    Chang Phueak  18.821  98.969
14         ต. สุเทพ             NaN  18.768  98.921


In [5]:
address = 'Chiang Mai City, CM'
##The geograpical coordinate of Chiang Mai City are 18.7743599, 98.9771944.
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chiang Mai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chiang Mai City are 18.7743599, 98.9771944.


In [6]:
# create map of Chiang Mai using latitude and longitude values
tambon_chiang_mai = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for LAT, LONG,TAMBON_E,TAMBON_T in zip(df['LAT'], df['LONG'], df['TAMBON_E'],df['TAMBON_T']):
    label = '{},{}'.format(TAMBON_E,TAMBON_T)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [LAT, LONG],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tambon_chiang_mai)  
    
tambon_chiang_mai

#### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'RWRXM2YICPLI44BR4KDCQPC2XFJEPXE0EDK3KOKJ2VG2INBM' # your Foursquare ID
CLIENT_SECRET = 'XIAHRN0AMFCHQ1CF44MMBCIFPEKNFNZXQ2BC55BLNUYID5UU' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 1000 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RWRXM2YICPLI44BR4KDCQPC2XFJEPXE0EDK3KOKJ2VG2INBM
CLIENT_SECRET:XIAHRN0AMFCHQ1CF44MMBCIFPEKNFNZXQ2BC55BLNUYID5UU


In [8]:
neighborhood_latitude=18.7883
neighborhood_longitude=98.9853

In [9]:
radius = 4000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&query=hotel&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=RWRXM2YICPLI44BR4KDCQPC2XFJEPXE0EDK3KOKJ2VG2INBM&client_secret=XIAHRN0AMFCHQ1CF44MMBCIFPEKNFNZXQ2BC55BLNUYID5UU&query=hotel&v=20180604&ll=18.7883,98.9853&radius=4000&limit=1000'

In [10]:
results = requests.get(url).json()


In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-12-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rachamankha Hotel,Hotel,18.787145,98.980127
1,Cozytel,Hotel,18.791398,98.986157
2,Le Méridien Chiang Mai,Hotel,18.784738,98.999714
3,U Chiang Mai Hotel,Hotel,18.788485,98.987138
4,Tamarind Village Hotel Chiang Mai,Hotel,18.788708,98.989820


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [47]:
# create map of New York using latitude and longitude values
map_chiang_mai = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(categories, name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chiang_mai)  
    
map_chiang_mai

## 4. Explore Neighborhoods in Manhattan

In [15]:
def getNearbyhotel(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&query=hotel&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['TAMBON_E', 
                  'TAMBON_E Latitude', 
                  'TAMBON_E Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
ChiangMai_Hotels = getNearbyhotel(names=df['TAMBON_E'],
                                   latitudes=df['LAT'],
                                   longitudes=df['LONG']
                                  )

Phra Sing
Hai Ya
Chang Moi
Pa Tan
Si Phum
San Phisuea
Nong Pa Khrang
Chang Khlan
Nong Hoi
Fa Ham
Wat Ket
Tha Sala
Pa Daet
Chang Phueak
nan


In [17]:
print(ChiangMai_Hotels.shape)
ChiangMai_Hotels.head(100)

(155, 7)


,TAMBON_E,TAMBON_E Latitude,TAMBON_E Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Phra Sing,18.785,98.985,makkachiva,18.785195,98.988624,Hotel
1,Phra Sing,18.785,98.985,U Chiang Mai Hotel,18.788485,98.987138,Hotel
2,Phra Sing,18.785,98.985,BED Phrasingh,18.787371,98.981441,Hotel
3,Phra Sing,18.785,98.985,Born2sleep,18.781084,98.985886,Motel
4,Phra Sing,18.785,98.985,Wing Bed,18.782400,98.981693,Hotel
5,Phra Sing,18.785,98.985,Chana Place,18.782861,98.982355,Hotel
6,Phra Sing,18.785,98.985,Gord Chiangmai (กอดเชียงใหม่),18.783475,98.985280,Bed & Breakfast
7,Phra Sing,18.785,98.985,Estia Chiang mai,18.789253,98.986259,Hotel
8,Phra Sing,18.785,98.985,BP Chiang Mai City Hotel,18.785928,98.983145,Hotel
9,Phra Sing,18.785,98.985,Namton Boutique Hotel,18.785864,98.982137,Hotel


In [18]:
ChiangMai_Hotels.groupby('TAMBON_E').count()

,TAMBON_E Latitude,TAMBON_E Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
TAMBON_E,,,,,,
Chang Khlan,7,7,7,7,7,7
Chang Moi,30,30,30,30,30,30
Chang Phueak,2,2,2,2,2,2
Fa Ham,1,1,1,1,1,1
Hai Ya,15,15,15,15,15,15
Nong Hoi,4,4,4,4,4,4
Nong Pa Khrang,1,1,1,1,1,1
Pa Daet,2,2,2,2,2,2
Pa Tan,3,3,3,3,3,3


In [19]:
print('There are {} uniques categories.'.format(len(ChiangMai_Hotels['Venue Category'].unique())))

There are 12 uniques categories.


## 5. Analyze Each Neighborhood(TAMBON_E)

In [20]:
# one hot encoding
ChiangMai_onehot = pd.get_dummies(ChiangMai_Hotels[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ChiangMai_onehot['TAMBON_E'] = ChiangMai_Hotels['TAMBON_E'] 

# move neighborhood column to the first column
fixed_columns = [ChiangMai_onehot.columns[-1]] + list(ChiangMai_onehot.columns[:-1])
ChiangMai_onehot = ChiangMai_onehot[fixed_columns]

ChiangMai_onehot.head(10)

,TAMBON_E,Bar,Bed & Breakfast,Café,Coffee Shop,Food,Hostel,Hotel,Hotel Pool,Motel,Residential Building (Apartment / Condo),Resort,Spa
0,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
1,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
2,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
3,Phra Sing,0,0,0,0,0,0,0,0,1,0,0,0
4,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
5,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
6,Phra Sing,0,1,0,0,0,0,0,0,0,0,0,0
7,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
8,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0
9,Phra Sing,0,0,0,0,0,0,1,0,0,0,0,0


In [21]:
ChiangMai_onehot.shape

(155, 13)

In [22]:
ChiangMai_grouped = ChiangMai_onehot.groupby('TAMBON_E').mean().reset_index()
ChiangMai_grouped

,TAMBON_E,Bar,Bed & Breakfast,Café,Coffee Shop,Food,Hostel,Hotel,Hotel Pool,Motel,Residential Building (Apartment / Condo),Resort,Spa
0,Chang Khlan,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.857143,0.000000,0.000000,0.000000,0.000000,0.000
1,Chang Moi,0.033333,0.100000,0.000000,0.000,0.000000,0.100000,0.700000,0.033333,0.033333,0.000000,0.000000,0.000
2,Chang Phueak,0.000000,0.500000,0.000000,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000
3,Fa Ham,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000
4,Hai Ya,0.000000,0.066667,0.000000,0.000,0.000000,0.200000,0.600000,0.000000,0.066667,0.000000,0.066667,0.000
5,Nong Hoi,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000
6,Nong Pa Khrang,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000
7,Pa Daet,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000
8,Pa Tan,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000
9,Phra Sing,0.000000,0.234043,0.021277,0.000,0.021277,0.063830,0.595745,0.000000,0.021277,0.042553,0.000000,0.000


In [23]:
ChiangMai_grouped.shape

(13, 13)

In [24]:
num_top_venues = 5

for hood in ChiangMai_grouped['TAMBON_E']:
    print("----"+hood+"----")
    temp = ChiangMai_grouped[ChiangMai_grouped['TAMBON_E'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chang Khlan----
             venue  freq
0            Hotel  0.86
1           Hostel  0.14
2              Bar  0.00
3  Bed & Breakfast  0.00
4             Café  0.00


----Chang Moi----
             venue  freq
0            Hotel  0.70
1  Bed & Breakfast  0.10
2           Hostel  0.10
3              Bar  0.03
4       Hotel Pool  0.03


----Chang Phueak----
             venue  freq
0  Bed & Breakfast   0.5
1            Hotel   0.5
2              Bar   0.0
3             Café   0.0
4      Coffee Shop   0.0


----Fa Ham----
             venue  freq
0            Hotel   1.0
1              Bar   0.0
2  Bed & Breakfast   0.0
3             Café   0.0
4      Coffee Shop   0.0


----Hai Ya----
             venue  freq
0            Hotel  0.60
1           Hostel  0.20
2  Bed & Breakfast  0.07
3            Motel  0.07
4           Resort  0.07


----Nong Hoi----
             venue  freq
0            Hotel   1.0
1              Bar   0.0
2  Bed & Breakfast   0.0
3             Café   0.0
4      Co

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['TAMBON_E']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['TAMBON_E'] = ChiangMai_grouped['TAMBON_E']

for ind in np.arange(ChiangMai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ChiangMai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Chang Khlan,Hotel,Hostel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food,Coffee Shop,Café
1,Chang Moi,Hotel,Hostel,Bed & Breakfast,Motel,Hotel Pool,Bar,Spa,Resort,Residential Building (Apartment / Condo),Food
2,Chang Phueak,Hotel,Bed & Breakfast,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop
3,Fa Ham,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
4,Hai Ya,Hotel,Hostel,Resort,Motel,Bed & Breakfast,Spa,Residential Building (Apartment / Condo),Hotel Pool,Food,Coffee Shop
5,Nong Hoi,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
6,Nong Pa Khrang,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
7,Pa Daet,Resort,Hotel,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
8,Pa Tan,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
9,Phra Sing,Hotel,Bed & Breakfast,Hostel,Residential Building (Apartment / Condo),Motel,Food,Café,Spa,Resort,Hotel Pool


## 6. Cluster Neighborhoods


In [27]:
# set number of clusters
kclusters = 5

ChiangMai_grouped_clustering = ChiangMai_grouped.drop('TAMBON_E', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ChiangMai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 4, 2, 1, 2, 2, 0, 2, 1], dtype=int32)

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ChiangMai_merged = df

# merge ChiangMai_grouped with df to add latitude/longitude for each TAMBON_E
ChiangMai_merged = ChiangMai_merged.join(neighborhoods_venues_sorted.set_index('TAMBON_E'), on='TAMBON_E')

ChiangMai_merged.head() # check the last columns!

,TAMBON_T,TAMBON_E,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ต. พระสิงห์,Phra Sing,18.785,98.985,1.0,Hotel,Bed & Breakfast,Hostel,Residential Building (Apartment / Condo),Motel,Food,Café,Spa,Resort,Hotel Pool
1,ต. หายยา,Hai Ya,18.777,98.984,1.0,Hotel,Hostel,Resort,Motel,Bed & Breakfast,Spa,Residential Building (Apartment / Condo),Hotel Pool,Food,Coffee Shop
2,ต. ช้างม่อย,Chang Moi,18.796,98.996,1.0,Hotel,Hostel,Bed & Breakfast,Motel,Hotel Pool,Bar,Spa,Resort,Residential Building (Apartment / Condo),Food
3,ต. ป่าตัน,Pa Tan,18.811,98.999,2.0,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
4,ต. ศรีภูมิ,Si Phum,18.796,98.985,1.0,Hotel,Hostel,Bed & Breakfast,Spa,Coffee Shop,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food


In [29]:
ChiangMai_merged

,TAMBON_T,TAMBON_E,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ต. พระสิงห์,Phra Sing,18.785,98.985,1.0,Hotel,Bed & Breakfast,Hostel,Residential Building (Apartment / Condo),Motel,Food,Café,Spa,Resort,Hotel Pool
1,ต. หายยา,Hai Ya,18.777,98.984,1.0,Hotel,Hostel,Resort,Motel,Bed & Breakfast,Spa,Residential Building (Apartment / Condo),Hotel Pool,Food,Coffee Shop
2,ต. ช้างม่อย,Chang Moi,18.796,98.996,1.0,Hotel,Hostel,Bed & Breakfast,Motel,Hotel Pool,Bar,Spa,Resort,Residential Building (Apartment / Condo),Food
3,ต. ป่าตัน,Pa Tan,18.811,98.999,2.0,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
4,ต. ศรีภูมิ,Si Phum,18.796,98.985,1.0,Hotel,Hostel,Bed & Breakfast,Spa,Coffee Shop,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food
5,ต. สันผีเสื้อ,San Phisuea,18.841,98.998,3.0,Resort,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hotel,Hostel,Food,Coffee Shop,Café
6,ต. หนองป่าครั่ง,Nong Pa Khrang,18.788,99.034,2.0,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
7,ต. ช้างคลาน,Chang Khlan,18.774,98.996,2.0,Hotel,Hostel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food,Coffee Shop,Café
8,ต. หนองหอย,Nong Hoi,18.758,99.015,2.0,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
9,ต. ฟ้าฮ่าม,Fa Ham,18.816,99.008,2.0,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café


In [30]:
ChiangMai_merged.dropna(subset = ['Cluster Labels'], inplace=True)
ChiangMai_merged['Cluster Labels'] =ChiangMai_merged['Cluster Labels'].astype(int)
ChiangMai_merged

,TAMBON_T,TAMBON_E,LAT,LONG,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ต. พระสิงห์,Phra Sing,18.785,98.985,1,Hotel,Bed & Breakfast,Hostel,Residential Building (Apartment / Condo),Motel,Food,Café,Spa,Resort,Hotel Pool
1,ต. หายยา,Hai Ya,18.777,98.984,1,Hotel,Hostel,Resort,Motel,Bed & Breakfast,Spa,Residential Building (Apartment / Condo),Hotel Pool,Food,Coffee Shop
2,ต. ช้างม่อย,Chang Moi,18.796,98.996,1,Hotel,Hostel,Bed & Breakfast,Motel,Hotel Pool,Bar,Spa,Resort,Residential Building (Apartment / Condo),Food
3,ต. ป่าตัน,Pa Tan,18.811,98.999,2,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
4,ต. ศรีภูมิ,Si Phum,18.796,98.985,1,Hotel,Hostel,Bed & Breakfast,Spa,Coffee Shop,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food
5,ต. สันผีเสื้อ,San Phisuea,18.841,98.998,3,Resort,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hotel,Hostel,Food,Coffee Shop,Café
6,ต. หนองป่าครั่ง,Nong Pa Khrang,18.788,99.034,2,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
7,ต. ช้างคลาน,Chang Khlan,18.774,98.996,2,Hotel,Hostel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food,Coffee Shop,Café
8,ต. หนองหอย,Nong Hoi,18.758,99.015,2,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
9,ต. ฟ้าฮ่าม,Fa Ham,18.816,99.008,2,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ChiangMai_merged['LAT'], ChiangMai_merged['LONG'], ChiangMai_merged['TAMBON_E'], ChiangMai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 7. Examine Clusters


#### Cluster 1

In [35]:
ChiangMai_merged.loc[ChiangMai_merged['Cluster Labels'] == 0, ChiangMai_merged.columns[[1] + list(range(5, ChiangMai_merged.shape[1]))]]

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Wat Ket,Resort,Hotel,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
12,Pa Daet,Resort,Hotel,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café


#### Cluster 2

In [37]:
ChiangMai_merged.loc[ChiangMai_merged['Cluster Labels'] == 1, ChiangMai_merged.columns[[1] + list(range(5, ChiangMai_merged.shape[1]))]]

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Phra Sing,Hotel,Bed & Breakfast,Hostel,Residential Building (Apartment / Condo),Motel,Food,Café,Spa,Resort,Hotel Pool
1,Hai Ya,Hotel,Hostel,Resort,Motel,Bed & Breakfast,Spa,Residential Building (Apartment / Condo),Hotel Pool,Food,Coffee Shop
2,Chang Moi,Hotel,Hostel,Bed & Breakfast,Motel,Hotel Pool,Bar,Spa,Resort,Residential Building (Apartment / Condo),Food
4,Si Phum,Hotel,Hostel,Bed & Breakfast,Spa,Coffee Shop,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food


#### Cluster 3

In [40]:
ChiangMai_merged.loc[ChiangMai_merged['Cluster Labels'] == 2, ChiangMai_merged.columns[[1] + list(range(5, ChiangMai_merged.shape[1]))]]

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Pa Tan,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
6,Nong Pa Khrang,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
7,Chang Khlan,Hotel,Hostel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Food,Coffee Shop,Café
8,Nong Hoi,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café
9,Fa Ham,Hotel,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop,Café


#### Cluster 4

In [42]:
ChiangMai_merged.loc[ChiangMai_merged['Cluster Labels'] == 3, ChiangMai_merged.columns[[1] + list(range(5, ChiangMai_merged.shape[1]))]]

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,San Phisuea,Resort,Spa,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hotel,Hostel,Food,Coffee Shop,Café


#### Cluster 5

In [45]:
ChiangMai_merged.loc[ChiangMai_merged['Cluster Labels'] == 4, ChiangMai_merged.columns[[1] + list(range(5, ChiangMai_merged.shape[1]))]]

,TAMBON_E,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Chang Phueak,Hotel,Bed & Breakfast,Spa,Resort,Residential Building (Apartment / Condo),Motel,Hotel Pool,Hostel,Food,Coffee Shop
